# CS145 Introduction to Data Mining - Assignment 5
## Deadline: 11:59PM (midnight), May 17, 2024

## Instructions
Each assignment is structured as a Jupyter notebook, offering interactive tutorials that align with our lectures. You will encounter two types of problems: *write-up problems* and *coding problems*.

1. **Write-up Problems:** These problems are primarily theoretical, requiring you to demonstrate your understanding of lecture concepts and to provide mathematical proofs for key theorems. Your answers should include sufficient steps for the mathematical derivations.
2. **Coding Problems:** Here, you will be engaging with practical coding tasks. These may involve completing code segments provided in the notebooks or developing models from scratch.

To ensure clarity and consistency in your submissions, please adhere to the following guidelines:

* For write-up problems, use Markdown bullet points to format text answers. Also, express all mathematical equations using $\LaTeX$ and avoid plain text such as `x0`, `x^1`, or `R x Q` for equations.
* For coding problems, comment on your code thoroughly for readability and ensure your code is executable. Non-runnable code may lead to a loss of **all** points. Coding problems have automated grading, and altering the grading code will result in a deduction of **all** points.
* Your submission should show the entire process of data loading, preprocessing, model implementation, training, and result analysis. This can be achieved through a mix of explanatory text cells, inline comments, intermediate result displays, and experimental visualizations.

### Collaboration and Integrity

* Collaboration is encouraged, but all final submissions must be your own work. Please acknowledge any collaboration or external sources used, including websites, papers, and GitHub repositories.
* Any suspicious cases of academic misconduct will be reported to The Office of the Dean of Students.

## Before You Start

Useful information about managing environments can be found [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

You may also quickly review the usage of basic Python and Numpy package, if needed in coding for matrix operations.

In this notebook, you must not delete any code cells in this notebook. If you change any code outside the blocks that you are allowed to edit (between `START/END YOUR CODE HERE`), you need to highlight these changes. You may add some additional cells to help explain your results and observations.

#### GPU Support

Considering the size of the training data, it is strongly suggested to use [Google Colab](https://colab.research.google.com/) or a GPU server for this exercise. If you are using Colab, you can manually switch to a CPU device on Colab by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`.

In [ ]:
import torch

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
elif USE_GPU and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
import scipy
import numpy as np
import networkx as nx

from torch import nn
from torch.nn import functional as F
from sklearn.cluster import KMeans

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

## Part 1: Variational Autoencoders (40 points)

## Brush-up of Information Theory

Many machine learning and deep learning methods are based on a variety of concepts from probability theory and information theory. 

Since Variational AutoEncoders are very intensive on this aspect, it can be useful to brush up some fundamentals.

If you are not already confident with the concepts of marginal probability, Shannon entropy, cross-entropy, Kullback–Leibler divergence and mutual information, we suggest you a fast reading, this [blog post](http://colah.github.io/posts/2015-09-Visual-Information/) with awesome visualizations by Chris Olah; and a book reference, [*Information Theory, Inference, and Learning Algorithms*](https://www.inference.org.uk/itprnn/book.pdf) by David J.C. MacKay.


**Reparameterization trick**

We need the reparameterization trick in order to backpropagate through a random node. 


Given:

In [ ]:
# Some data
data = torch.rand(10)

# Some parameters to train
deterministic_transform1 = torch.nn.Linear(in_features=10,  out_features=10)
deterministic_transform2 = torch.nn.Linear(in_features=10,  out_features=10)

# My expected output
gold_output = torch.ones(10)

We want optimize the parameters in order to obtain `gold_output` from the following computation:

In [ ]:
# My stochastic prediction
pred_mean = deterministic_transform1(data)
pred_std = torch.ones_like(data)

sampling = torch.normal(mean=pred_mean, std=pred_std)     # random numbers drawn from **separate** normal distributions
                                                          # whose mean and standard deviation are given.

pred_output = deterministic_transform2(sampling)

# Error function
loss = F.mse_loss(pred_output, gold_output)
deterministic_transform1.zero_grad()
loss.backward()

# then optimize with some optimizer

But...

In [ ]:
print(loss)
print(deterministic_transform1.weight.grad  )

We have a problem, this doesn’t work! We don’t have any gradients! They are zero even though the loss is non-zero



The specific problem here is that we can’t backpropagate through the random `normal` function. 

This makes sense right? We are trying to back propagate through a random/stochastic node in the computational graph. 
It doesn’t make much sense to differentiate with respect to a stochastic node since that means the gradient would technically be a random variable too!

So let’s... reparametrize! That is, let’s change how the parameters are incorporated into the model.

![](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/08/trick.png)


### Problem 1 (10 points)

Implement the `reparameterize` function that takes the mean `mu` and log-variance `logvar` as input.
In this function, compute the standard deviation from the log-variance using the exponential function.
Then, sample a random noise from a standard normal distribution with the same shape as the standard deviation.
Finally, compute the sampled latent vector `z` using the reparameterization trick.
Return the sampled latent vector `z`.

Note that here using the log-variance instead of the standard deviation improves numerical stability during training.
The log-variance is used to compute the standard deviation by applying the exponential function: `std = exp(0.5 * logvar)`.

In [ ]:
def reparameterize(mu, logvar):
    """
    Reparameterization trick to sample from N(mu, var) from N(0, 1).

    Args:
        mu: Mean of the latent Gaussian [batch_size, latent_dim]
        logvar: Log variance of the latent Gaussian [batch_size, latent_dim]

    Returns:
        z: Sampled latent vector [batch_size, latent_dim]
    """

    eps = torch.randn_like(mu)  # Sample random noise from a standard normal distribution
    # START YOUR CODE HERE
    std = None  # Compute the standard deviation from the log-variance
    z = None  # Compute the sampled latent vector using the reparameterization trick
    # END YOUR CODE HERE

    return z

In [ ]:
# Test case for reparameterize function
mu = torch.tensor([[0.0, 0.0]])
logvar = torch.tensor([[1.0, 1.0]])
z = reparameterize(mu, logvar)
assert z.shape == (1, 2), "Output shape of reparameterize should be (1, 2)"

In [ ]:
# My stochastic prediction
pred_mean = deterministic_transform1(data)
pred_std = torch.ones_like(data)

rand_source = torch.randn(10)                       # random numbers drawn from **separate** canonical normal distributions
sampling = reparameterize(pred_mean, pred_std)

pred_output = deterministic_transform2(sampling)

# My error
loss = F.mse_loss(pred_output, gold_output)
deterministic_transform1.zero_grad()
loss.backward()

# then optimize with some optimizer

In [ ]:
deterministic_transform1.weight.grad

That's it, now we can adjust the parameters of the distribution to improve our stochastic prediction!

**We constructed the parameterized random variable via a parameterized deterministic function of a parameter-free random variable.**



Another way of thinking about it is that we have moved the source of noise outside of the main flow of the network and used the noise as a way to sample from the expectation.

For a formal explanation check out this [post](https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important).

### Problem 2 (10 points)

Let $x$ be the input, $z$ the latent variable, $θ$ the decoder parameters, and $φ$ the encoder parameters.
Show that the VAE training objective $\log p_\theta(x)$ consists of two terms:
1. Reconstruction loss
2. KL divergence between the encoded latent distribution $q_φ(z\mid x)$ and $p(z)$, the prior distribution 

**[TODO: Write your answer here]**

The Variational Autoencoder (VAE) training objective aims to maximize the log-likelihood of the observed data $ \log p_\theta(x) $. This objective can be decomposed into two terms: the reconstruction loss and the KL divergence between the encoded latent distribution $ q_\phi(z|x) $ and the prior distribution $ p(z) $. Here's a step-by-step derivation:

### VAE Objective: Evidence Lower Bound (ELBO)

Given an input $ x $, the log-likelihood $ \log p_\theta(x) $ can be decomposed using the variational distribution $ q_\phi(z|x) $:

$ \log p_\theta(x) = \log \int p_\theta(x, z) \, dz $

Since the integral is generally intractable, we introduce the variational distribution $ q_\phi(z|x) $ and use Jensen's inequality to derive a lower bound:

$ \log p_\theta(x) \geq \mathbb{E}_{q_\phi(z|x)} \left[ \log \frac{p_\theta(x, z)}{q_\phi(z|x)} \right] $

This lower bound is called the Evidence Lower Bound (ELBO):

$ \log p_\theta(x) \geq \mathbb{E}_{q_\phi(z|x)} \left[ \log p_\theta(x|z) \right] - \text{KL}(q_\phi(z|x) \| p(z)) $

### Decomposition into Reconstruction Loss and KL Divergence

Let's break down the terms in the ELBO:

1. **Reconstruction Loss**:
   $ \mathbb{E}_{q_\phi(z|x)} \left[ \log p_\theta(x|z) \right] $

   This term represents the expected log-likelihood of the data given the latent variable $ z $. It measures how well the decoder $ p_\theta(x|z) $ can reconstruct the input $ x $ from the sampled latent variable $ z $.

2. **KL Divergence**:
   $ \text{KL}(q_\phi(z|x) \| p(z)) = \mathbb{E}_{q_\phi(z|x)} \left[ \log \frac{q_\phi(z|x)}{p(z)} \right] $

   This term measures the divergence between the approximate posterior $ q_\phi(z|x) $ and the prior $ p(z) $. It acts as a regularizer, ensuring that the learned latent distribution does not deviate too far from the prior.

### Intuition Behind the Terms

- **Reconstruction Loss**: Encourages the model to reconstruct the input $ x $ accurately from the latent variable $ z $. A lower reconstruction loss indicates better reconstruction quality.

- **KL Divergence**: Encourages the learned latent distribution $ q_\phi(z|x) $ to be close to the prior distribution $ p(z) $. This regularization helps to prevent overfitting and ensures that the latent space has a meaningful structure.

### Final VAE Objective

Combining the two terms, the ELBO can be rewritten as:

$ \log p_\theta(x) \geq \mathbb{E}_{q_\phi(z|x)} \left[ \log p_\theta(x|z) \right] - \text{KL}(q_\phi(z|x) \| p(z)) $

Maximizing this lower bound (ELBO) during training is equivalent to minimizing the reconstruction loss and the KL divergence.

### Summary

The VAE training objective $ \log p_\theta(x) $ consists of two terms:

1. **Reconstruction Loss**: $ \mathbb{E}_{q_\phi(z|x)} \left[ \log p_\theta(x|z) \right] $, which measures how well the decoder can reconstruct the input.
2. **KL Divergence**: $ \text{KL}(q_\phi(z|x) \| p(z)) $, which regularizes the learned latent distribution to be close to the prior distribution.

These two terms together ensure that the VAE learns a meaningful latent representation while being able to generate realistic data.

### Problem 3 (10 points)

Instructions:
- The `VAE` model class contains an encoder and a decoder.
- The encoder outputs the mean `mu` and log-variance `logvar` of the latent Gaussian distribution.
- The decoder reconstructs the input data from the sampled latent vector.
- Implement the `forward` method to perform the encoding, sampling, and decoding process.
- Implement the `loss_function` method to compute the VAE loss.

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=64, latent_dim=20):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=hidden_channels, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_channels, out_channels=hidden_channels*2, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(hidden_channels*2*7*7, latent_dim*2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_channels*2*7*7),
            nn.ReLU(),
            nn.Unflatten(1, (hidden_channels*2, 7, 7)),
            nn.ConvTranspose2d(in_channels=hidden_channels*2, out_channels=hidden_channels, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=hidden_channels, out_channels=input_channels, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

        self.latent_dim = latent_dim

    def encode(self, x):
        """
        Encodes the input into parameters of a latent Gaussian distribution.
        """
        x = self.encoder(x)
        mu, logvar = x[:, :self.latent_dim], x[:, self.latent_dim:]
        return mu, logvar

    def decode(self, z):
        """
        Decodes the latent vector into reconstructed input data.
        """
        return self.decoder(z)

    def forward(self, x):
        # START YOUR CODE HERE
        mu, log_var = None  # Encode the input to obtain mean and log-variance of the latent Gaussian
        z = None  # Sample the latent vector using the reparameterization trick
        recon_x = None  # Decode the latent vector to obtain the reconstructed input
        # END YOUR CODE HERE
        return recon_x, mu, log_var

    def loss_function(self, recon_x, x, mu, log_var):
        """
        Computes the VAE loss function.
        """
        # START YOUR CODE HERE
        recon_loss = None  # Reconstruction loss
        kl_div = None  # KL divergence loss
        loss = None  # Total VAE loss
        # END YOUR CODE HERE
        return loss

**Encoder and Decoder**


We'll use a convolutional encoder and decoder, which generally gives better performance than fully connected versions that have the same number of parameters.

**Bottleneck**

In the convolution layers, we increase the channels as we approach the bottleneck, but note that the total number of features still decreases, since the channels increase by a factor of 2 in each convolution, but the spatial size decreases by a factor of 4.


**Deconvolution**

In the decoder we are using a transposed convolution, also known as deconvolution. You can think of it as the convolution in the opposite direction.  ([here](https://github.com/vdumoulin/conv_arithmetic) more visualizations)

*Blue maps are inputs, and cyan maps are outputs*

![](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/08/deconv.gif)



**Kernel size**

We are using `kernel_size=4`. The motivation behind this choice is to lessen the checkerboard artifacts described [here](https://distill.pub/2016/deconv-checkerboard)

### Problem 4 (10 points)

Instructions:
- Train the VAE model on the MNIST dataset for a specified number of epochs.
- Plot the training loss over epochs.
- Visualize the learned latent space by sampling points from the latent space and decoding them.
- Analyze the structure of the latent space and discuss your observations.

In [ ]:
num_epochs = 50
batch_size = 128
learning_rate = 1e-3

dataset = MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = VAE(hidden_channels=64, latent_dim=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_history = []
for epoch in range(num_epochs):
    for batch_idx, (data, _) in enumerate(data_loader):
        data = data.to(device)
        recon_batch, mu, log_var = model(data)
        loss = model.loss_function(recon_batch, data, mu, log_var)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss_history.append(loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Plot the training loss
plt.plot(loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
with torch.no_grad():
    # Sample latent vectors from the prior distribution
    prior_samples = torch.randn(64, 2).to('mps')  # Sample 64 points from the prior distribution
    generated_samples = model.decode(prior_samples).view(64, 1, 28, 28).detach().cpu()
    
    plt.figure(figsize=(8, 8))
    for i in range(64):
        plt.subplot(8, 8, i+1)
        plt.imshow(generated_samples[i].squeeze(), cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
with torch.no_grad():
    for _ in range(5):
        # Select two random images from the test set
        img1, _ = dataset[np.random.randint(len(dataset))]
        img2, _ = dataset[np.random.randint(len(dataset))]

        # Encode the images into latent vectors
        z1 = model.encode(img1.view(1, 1, 28, 28).to(device))[0]
        z2 = model.encode(img2.view(1, 1, 28, 28).to(device))[0]

        # Interpolate between the latent vectors
        interpolations = []
        for alpha in np.linspace(0, 1, 20):
            z_interp = alpha * z1 + (1 - alpha) * z2
            interpolations.append(model.decode(z_interp).view(1, 28, 28))

        interpolations = torch.cat(interpolations)

        plt.figure(figsize=(12, 4))
        for i in range(20):
            plt.subplot(2, 10, i+1)
            plt.imshow(interpolations[i].squeeze().detach().cpu(), cmap='gray')
            plt.axis('off')
        plt.suptitle('Latent Space Interpolation')
        plt.show()

**[TODO: Write your answer here]**

### Latent space (from Jeremy Jordan's [blog post](https://www.jeremyjordan.me/variational-autoencoders/))

Using VAEs we are able to learn **smooth latent state representations** of the input data. For standard autoencoders, we simply need to learn an encoding which allows us to reproduce the input. 

As you can see in the left-most figure, focusing only on reconstruction loss does allow us to separate out the classes (in this case, MNIST digits) which should allow our decoder model the ability to reproduce the original handwritten digit, but there's an uneven distribution of data within the latent space. In other words, there are areas in latent space which don't represent any of our observed data.

![](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/08/latent.png)



On the flip side, if we only focus only on ensuring that the latent distribution is similar to the prior distribution (through our KL divergence loss term), we end up describing every observation using the same unit Gaussian, which we subsequently sample from to describe the latent dimensions visualized. This effectively treats every observation as having the same characteristics; in other words, we've failed to describe the original data.

However, when the two terms are optimized simultaneously, we're encouraged to describe the latent state for an observation with distributions close to the prior but deviating when necessary to describe salient features of the input.

![](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/08/distr.png)

#### 2D Latent Space

The figure below visualizes the data generated by the decoder network of a variational autoencoder trained on the MNIST handwritten digits dataset. 

We are doing a linear interpolation over a (multivariate) gaussian distribution!

Here, we sample and decode latent space coordinates proportionally to the model’s distribution over latent space. In other words, we simply sample relative to our chosen prior distribution over z
. In our case, this means sampling linearly spaced percentiles from the [inverse CDF](http://work.thaslwanter.at/Stats/html/statsDistributions.html#other-important-presentations-of-probability-densities) of a spherical Gaussian.

In [ ]:
import torchvision
import matplotlib.pyplot as plt

latents_lims = 3.66
num_interpolations = 30

nd = torch.distributions.Normal(loc=torch.as_tensor([0.]),
                                scale=torch.as_tensor([1.]))

with torch.no_grad():
    latent_interpolation = torch.linspace(0.001, 0.999, num_interpolations)
    latent_grid = torch.stack(
        (
            latent_interpolation.repeat(num_interpolations, 1),
            latent_interpolation[:, None].repeat(1, num_interpolations)
        ), dim=-1).view(-1, 2)

    latent_grid = nd.icdf(latent_grid)
    plt.scatter(latent_grid[:,0].cpu().numpy(), latent_grid[:,1].cpu().numpy())
    plt.show()
    # reconstruct images from the latent vectors
    latent_grid = latent_grid.to(device)
    image_recon = model.decoder(latent_grid)
    image_recon = image_recon.cpu()

    # Matplolib plot, much faster for static images
    plt.figure(figsize = (17, 17))
    plt.imshow(torchvision.utils.make_grid(
        image_recon.data[:num_interpolations ** 2], 
        num_interpolations).permute(1, 2, 0))
    plt.title("2D latent space")
    plt.axis('off')
    plt.show()

## Part 2: Node Embeddings (50 points)

The Karate Club Network is a graph describes a social network of 34 members of a karate club and documents links between members who interacted outside the club.

In [ ]:
G = nx.karate_club_graph()

# G is an undirected graph
print(type(G))

# Visualize the graph
nx.draw(G, with_labels = True)

### Problem 5 (5 points)

Compute the average degree of the karate club network.

Instructions:
- Implement the `average_degree` function that takes the number of edges and nodes as input.
- Compute the average degree of the graph.
- Round the result to the nearest integer. For example, 3.3 will be rounded to 3 and 3.7 will be rounded to 4.

In [ ]:
def average_degree(num_edges, num_nodes):
    """
    Args:
        num_edges: Number of edges in the graph
        num_nodes: Number of nodes in the graph

    Returns:
        avg_degree: Average degree of the graph
    """
    # START YOUR CODE HERE
    avg_degree = None  # Compute the average degree
    # END YOUR CODE HERE
    return avg_degree

In [ ]:
num_edges = G.number_of_edges()
num_nodes = G.number_of_nodes()
print("Num edges:", num_edges, "Num nodeds:", num_nodes)
avg_degree = average_degree(num_edges, num_nodes)
print("Average degree of karate club network is {}".format(avg_degree))

### Problem 6 (5 points)

Implement functions to compute the Laplacian and Random Walk matrices.

Instructions:
- Implement the `laplacian_matrix` function that takes the adjacency matrix as input and returns the Laplacian matrix.
- Implement the `random_walk_matrix` function that takes the adjacency matrix as input and returns the random walk matrix.

In [ ]:

def laplacian_matrix(A):
    """
    Args:
        A: Adjacency matrix of the graph

    Returns:
        L: Laplacian matrix
    """
    # START YOUR CODE HERE
    D = None  # Compute the degree matrix
    L = None  # Compute the Laplacian matrix
    # END YOUR CODE HERE
    return L

def random_walk_matrix(A):
    """
    Args:
        A: Adjacency matrix of the graph

    Returns:
        P: Random walk matrix
    """
    # START YOUR CODE HERE
    D = None  # Compute the degree matrix
    P = None  # Compute the random walk matrix
    # END YOUR CODE HERE
    return P

### Problem 7 (5 points)

Compute the PageRank value for node 0 after one PageRank iteration:

$$
r_j = \sum_{i \rightarrow j} \beta \frac{r_i}{d_i} + (1 - \beta) \frac{1}{N}
$$

Instructions:
- Implement the `one_iter_pagerank` function that takes the graph, damping factor, initial PageRank value, and node ID as input.
- Compute the PageRank value for the specified node after one iteration.
- Round the result to 2 decimal places.

In [ ]:
def one_iter_pagerank(G, beta, r0, node_id):
    """
    Args:
        G: NetworkX graph
        beta: Damping factor
        r0: Initial PageRank value for each node
        node_id: Node to compute PageRank for

    Returns:
        r1: PageRank value after one iteration
    """
    # START YOUR CODE HERE
    r1 = None
    # END YOUR CODE HERE
    return r1

In [ ]:
beta = 0.8
r0 = 1 / G.number_of_nodes()
node = 0
r1 = one_iter_pagerank(G, beta, r0, node)
print("The PageRank value for node 0 after one iteration is {}".format(r1))

### Problem 8 (5 points)

Compute the final PageRank values using the random walk matrix.

Instructions:
- Implement the `pagerank` function that takes the random walk matrix, number of iterations, and damping factor as input.
- Compute the final PageRank values using the power iteration method.
- Compare your results with the PageRank values obtained from NetworkX.


In [ ]:
def pagerank(P, num_iterations=100, d=0.85):
    """
    Args:
        P: Random walk matrix
        num_iterations: Number of iterations
        d: Damping factor

    Returns:
        r: Final PageRank values
    """
    # START YOUR CODE HERE
    r = None
    # END YOUR CODE HERE
    return r

In [ ]:
# Test case for pagerank function
G = nx.karate_club_graph()
P = random_walk_matrix(nx.adjacency_matrix(G).todense())
damping_factor = 0.85
num_iterations = 100

# Compute PageRank using the implemented pagerank function
pagerank_scores = pagerank(P, num_iterations, damping_factor)

# Compute PageRank using the NetworkX implementation
nx_pagerank_scores = nx.pagerank(G, alpha=damping_factor, max_iter=num_iterations)

# Convert NetworkX PageRank scores to a numpy array
nx_pagerank_scores = np.array(list(nx_pagerank_scores.values()))

print("Computed PageRank scores:")
print(pagerank_scores)

print("NetworkX PageRank scores:")
print(nx_pagerank_scores)

# Compare the computed PageRank scores with the NetworkX scores
assert np.allclose(pagerank_scores, nx_pagerank_scores, rtol=1e-3), "Computed PageRank scores do not match the NetworkX scores"

In [ ]:
# Visualizing the graph with node sizes proportional to PageRank scores
node_sizes_custom = [20000 * pagerank_scores[i] for i in range(len(pagerank_scores))]  # Scale factor for visibility
node_sizes_nx = [20000 * nx_pagerank_scores[node] for node in G.nodes()]

plt.figure(figsize=(12, 8))
plt.subplot(121)
nx.draw(G, node_size=node_sizes_custom, with_labels=True, font_weight='bold')
plt.title("Your PageRank")

plt.subplot(122)
nx.draw(G, node_size=node_sizes_nx, with_labels=True, font_color='white', node_color='blue')
plt.title("NetworkX PageRank")

plt.show()

### Problem 9 (10 points)
Implement negative edge sampling.

Instructions:
- Implement the `sample_negative_edges` function that takes a NetworkX graph and the number of negative samples as input.
- Sample negative edges that do not exist in the graph.
- Return the list of sampled negative edges.

Notes: You do not need to consider the corner case when the number of possible negative edges
is less than `num_neg_samples`. It should be ok as long as your implementation 
works on the karate club network. In this implementation, self loop should 
not be considered as either a positive or negative edge. Also, notice that 
the karate club network is an undirected graph, if (0, 1) is a positive 
edge, do you think (1, 0) can be a negative one?

In [ ]:
def graph_to_edge_list(G):
    """
    Args:
        G: NetworkX graph

    Returns:
        edge_list: List of edges in the graph
    """
    edge_list = []
    for edge in G.edges():
        edge_list.append(edge)  # Append each edge to the edge list
    return edge_list

def edge_list_to_tensor(edge_list):
    """
    Args:
        edge_list: List of edges in the graph

    Returns:
        edge_index: Tensor of shape (2, num_edges)
    """
    edge_index = torch.tensor(edge_list).t().contiguous()  # Convert the edge list to a tensor and transpose it
    return edge_index


pos_edge_list = graph_to_edge_list(G)
pos_edge_index = edge_list_to_tensor(pos_edge_list)
print("The pos_edge_index tensor has shape {}".format(pos_edge_index.shape))
print("The pos_edge_index tensor has sum value {}".format(torch.sum(pos_edge_index)))

def sample_negative_edges(G, num_neg_samples):
    """
    Args:
        G: NetworkX graph
        num_neg_samples: Number of negative examples to sample

    Returns:
        neg_edge_list: List of negative edges
    """
    # START YOUR CODE HERE
    neg_edge_list = []
    # END YOUR CODE HERE
    return neg_edge_list

In [ ]:
# Sample 78 negative edges
neg_edge_list = sample_negative_edges(G, len(pos_edge_list))

# Transform the negative edge list to tensor
neg_edge_index = edge_list_to_tensor(neg_edge_list)
print("The neg_edge_index tensor has shape {}".format(neg_edge_index.shape))

# Which of following edges can be negative ones?
edge_1 = (7, 1)
edge_2 = (1, 33)
edge_3 = (33, 22)
edge_4 = (0, 4)
edge_5 = (4, 2)

## Note:
## 1: For each of the 5 edges, print whether it can be negative edge
pos_edge_list = graph_to_edge_list(G)
for edge in [edge_1, edge_2, edge_3, edge_4, edge_5]:
    edge = (edge[1], edge[0]) if edge[0] > edge[1] else edge
    if edge in pos_edge_list:
        print("No")
    else:
        print("Yes")

### Problem 10 (15 points)

Train the node embedding model and report the best performance.

Instructions:
- Implement the `forward` method to compute the dot product of the embeddings for each edge.
- train the node embedding model using positive and negative edges.
- Use binary cross-entropy loss and optimize the model using Adam optimizer.
- Train the model for a specified number of epochs and report the best loss and accuracy achieved.
- Plot the training loss over epochs.
- Visualize the learned node embeddings in a 2D space using a scatter plot.

In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, num_nodes, embedding_dim):
        super(EmbeddingModel, self).__init__()
        self.embedding = nn.Embedding(num_nodes, embedding_dim)  # Initialize the embedding layer

    def forward(self, edge):
        # START YOUR CODE HERE
        emb = None  # Get the embeddings for the nodes in the edge
        prod = None  # Compute the element-wise product of the embeddings
        out = None  # Sum the element-wise product to get the dot product
        # END YOUR CODE HERE
        return out

def train(model, pos_edge_index, neg_edge_index, num_epochs=500):
    """
    Args:
        model: Node embedding model
        pos_edge_index: Tensor of positive edges
        neg_edge_index: Tensor of negative edges
        num_epochs: Number of epochs to train for

    Returns:
        loss_history: List of training losses over epochs
        best_loss: Best loss achieved during training
        best_accuracy: Best accuracy achieved during training
        model: Trained model
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    pos_label = torch.ones(pos_edge_index.shape[1], dtype=torch.float)
    neg_label = torch.zeros(neg_edge_index.shape[1], dtype=torch.float)
    train_label = torch.cat([pos_label, neg_label], dim=0)
    train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

    loss_history = []
    best_loss = float('inf')
    best_accuracy = 0.0
    loss_fn = nn.BCEWithLogitsLoss()  # Define the binary cross-entropy loss function

    for epoch in range(num_epochs):
        optimizer.zero_grad()  # Reset the gradients
        out = model(train_edge)  # Forward pass through the model
        
        loss = loss_fn(out, train_label)  # Compute the loss for the current epoch
        loss_history.append(loss.item())  # Append the loss to the list of losses

        loss.backward()  # Compute the gradients
        optimizer.step()  # Update the model parameters

        # Compute the accuracy
        pred = torch.sigmoid(out)
        pred_label = (pred > 0.5).float()
        accuracy = (pred_label == train_label).float().mean()

        # Update the best loss and accuracy
        if loss < best_loss:
            best_loss = loss.item()
            best_accuracy = accuracy.item()

    return loss_history, best_loss, best_accuracy, model

In [ ]:
G = nx.karate_club_graph()
pos_edge_list = graph_to_edge_list(G)
pos_edge_index = edge_list_to_tensor(pos_edge_list)
neg_edge_list = sample_negative_edges(G, len(pos_edge_list))
neg_edge_index = edge_list_to_tensor(neg_edge_list)

model = EmbeddingModel(G.number_of_nodes(), embedding_dim=16)
loss_history, best_loss, best_accuracy, model = train(model, pos_edge_index, neg_edge_index, num_epochs=500)

print(f"Best loss: {best_loss:.4f}")
print(f"Best accuracy: {best_accuracy:.4f}")

# Plot the training loss
plt.figure()
plt.plot(loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
from sklearn.decomposition import PCA

X = model.embedding.weight.data.numpy()
pca = PCA(n_components=2)
components = pca.fit_transform(X)
plt.figure(figsize=(6, 6))
club1_x = []
club1_y = []
club2_x = []
club2_y = []
for node in G.nodes(data=True):
    if node[1]['club'] == 'Mr. Hi':
        club1_x.append(components[node[0]][0])
        club1_y.append(components[node[0]][1])
    else:
        club2_x.append(components[node[0]][0])
        club2_y.append(components[node[0]][1])
plt.scatter(club1_x, club1_y, color="red", label="Mr. Hi")
plt.scatter(club2_x, club2_y, color="blue", label="Officer")
plt.legend()
plt.show()

## Part 3: Spectral Clustering (10 points)

### Problem 11 (10 points)
Implement symmetric normalized spectral clustering.

Instructions:
- Implement the `spectral_clustering_sym` function that takes the adjacency matrix and the number of clusters as input.
- Compute the symmetric normalized Laplacian matrix.
- Compute the eigenvectors corresponding to the smallest eigenvalues of the normalized Laplacian matrix.
- Apply K-means clustering on the eigenvectors to obtain the cluster labels.

In [ ]:
def spectral_clustering_rw(A, k):
    """
    Args:
        A: Adjacency matrix
        k: Number of clusters
    Returns:
        labels: Cluster labels
    """
    # START YOUR CODE HERE
    D = None  # Compute the degree matrix
    L = None  # Compute the unnormalized Laplacian matrix
    Dinv = None  # Compute the inverse of the degree matrix
    Lrw = None  # Compute the random walk normalized Laplacian matrix

    # Compute the eigenvectors corresponding to the k smallest eigenvalues
    # Apply K-means clustering on the eigenvectors
    labels = None # Obtain the cluster labels
    # END YOUR CODE HERE
    return labels

In [ ]:
G = nx.karate_club_graph()
A = nx.adjacency_matrix(G).todense()

labels_rw = spectral_clustering_rw(A, 4)

print(f"Random walk normalized Laplacian labels: {labels_rw}")

In [ ]:
# Visualize the clusterings
pos = nx.spring_layout(G)


nx.draw_networkx(G, pos, node_color=labels_rw)
plt.title('Random Walk Normalized Laplacian') 

plt.tight_layout()
plt.show()